# Retake exam: 20 June 2023

## Topic of the exam: Understand top players of global land use and CO2 emissions 

Please ensure that you run each cell (ctrl+enter) so that your inputs are saved 

All questions have either coding cell or a text cell, or a combination of the two. 

You are allowed to create additional cells for each answer to facilitate your work  

## Data preparation: Download and import exiobase

Data to be used in the exam: **year 2019 product by product format**

Population data to be downloaded together with the script

Link to the data: https://zenodo.org/record/5589597

In [8]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import MRIO variables
path = "data/IOT_2019_pxp/" 
A = pd.read_csv(f'{path}A.txt', sep='\t', index_col=[0, 1], header=[0, 1])
Y = pd.read_csv(f'{path}Y.txt' , sep='\t', index_col=[0, 1], header=[0, 1])

# Import Impact accounts
F_sat = pd.read_csv(f'{path}impacts/F.txt' , sep='\t', index_col=[0], header=[0, 1])
F_sat_hh = pd.read_csv(f'{path}impacts/F_Y.txt' , sep='\t', index_col=[0], header=[0, 1])

# Create labels
A_labels = A.index.to_frame(index=None)

region_labels = A_labels.region.drop_duplicates().reset_index(drop=True)
sector_labels = A_labels.sector.drop_duplicates().reset_index(drop=True)

r = region_labels.shape[0]
s = sector_labels.shape[0]

# Aggregate Y and F_hh by region
Y_reg = Y.groupby(level=0, axis=1, sort=False).sum()
F_sat_hh_reg = F_sat_hh.groupby(level=0, axis=1, sort=False).sum()

# Aggregate Y and F_hh by final demand category
Y_fd = Y.groupby(level=1, axis=1, sort=False).sum()
F_sat_hh_fd = F_sat_hh.groupby(level=1, axis=1, sort=False).sum()

In [3]:
#Calculate rest of MRIO variables
I = np.identity(A.shape[0])
L = np.linalg.inv(I-A)
x = L @ Y.sum(axis=1)

x_ = x.copy()
x_[x_!=0] = 1/x_[x_!=0]
inv_diag_x_ = np.diag(x_)

## Question 1: Territorial accounting (10 points)

### Q1.1 Can you identify the top 3 countries with the highest per capita domestic land use in 2019? (10 points) 

Hint 1: use the environmental impact with keywords: "Land use Crop, Forest, Pasture"

Hint 2: when loc returns a DataFrame instead of a Series you can use .squeeze() to obtain the Series

In [4]:
#import population data
population = pd.read_excel('data/EXIOBASE_PopulationGDP_1995_2019.xlsx',sheet_name='Population', index_col=[0, 1, 2])
pop2019 = population.loc[:,"y2019"]

In [6]:
#land use extensions
f_sat_land_use = F_sat[F_sat.index.str.contains("Land use Crop, Forest, Pasture")].sum(axis=0)
F_sat_hh_land_use = F_sat_hh[F_sat_hh.index.str.contains("Land use Crop, Forest, Pasture")].sum(axis=0)

F_sat_hh_land_use_reg = F_sat_hh_land_use.groupby(level=0, sort=False).sum()

e_land_use_pba = np.diag(f_sat_land_use) @ x
e_land_use_ter = e_land_use_pba.reshape(r,s).sum(1) + F_sat_hh_land_use_reg

e_land_use_ter_pp = e_land_use_ter/pop2019.values

e_land_use_ter_pp.nlargest(n=3)

region
US    994.906327
CA    726.252900
AU    651.702384
dtype: float64

## Question 2: Footprint analysis (10 points)

### Q2.1 Which 3 regions had the highest per capita footprint of land use in 2019? (10 points)

In [9]:
e_land_use_reg = f_sat_land_use @ L @ Y_reg + F_sat_hh_land_use_reg 
e_land_use_reg_pp = e_land_use_reg/pop2019.values
e_land_use_reg_pp.nlargest(n=3)

region
LU    1496.661642
US     958.905948
CA     630.142254
dtype: float64

## Questions 3: Trade (15 points)

### Q3.1: Were the regions found in Q1 and Q2 net importers or net exporter of per capita land use in 2019? (3 points)

In [22]:
e_net_import = e_land_use_reg - e_land_use_ter
e_net_import_pp_land_use = e_net_import/pop2019.values
e_net_import_pp_land_use.loc[['LU', 'US', 'CA', 'AU']]

region
LU    1496.075170
US     -36.000380
CA     -96.110646
AU    -282.396448
dtype: float64

_Write your answer here_

### Q3.2: Taking into account all exiobase regions, perform MRIO calculations and identify the top three regions that were net per capita importers of land use in 2019 (12 points)

In [15]:
E_land_use_=np.diag(f_sat_land_use) @ L @ Y_reg
E_land_use_.index = Y.index

E_land_use_top_3 = E_land_use_/pop2019.values
E_land_use_top_3.sum().nlargest(n=3)

region
LU    1496.661479
US     958.904294
CA     630.141450
dtype: float64

## Question 4: Dashboard analysis - Land Use vs. CO2 emissions (25 points)

### Q4.1 Identify global regions with the highest total land use and compare them to their CO2 emissions per capita and Land use per capita. (10 points)

Hint 1: Replicate Qs 2-3 for CO2 emissions

Hint 2: use the environmental extensions with keywords: "CO2 - combustion - air"

In [18]:
# Import Satellite accounts
F_sat = pd.read_csv(f'{path}satellite/F.txt' , sep='\t', index_col=[0], header=[0, 1])
F_sat_hh = pd.read_csv(f'{path}satellite/F_Y.txt' , sep='\t', index_col=[0], header=[0, 1])

#countreis with highest per capita domestic CO2 consumption
f_sat_CO2 = F_sat[F_sat.index.str.contains("CO2 - combustion - air")].sum(axis=0)
F_sat_hh_CO2 = F_sat_hh[F_sat_hh.index.str.contains("CO2 - combustion - air")].sum(axis=0)

F_sat_hh_CO2_reg = F_sat_hh_CO2.groupby(level=0, sort=False).sum()

e_CO2_pba = np.diag(f_sat_CO2) @ x
e_CO2_ter = e_CO2_pba.reshape(r,s).sum(1) + F_sat_hh_CO2_reg

e_CO2_ter_pp = e_CO2_ter/pop2019.values/1000

e_CO2_ter_pp.nlargest(n=3)


region
CN    3.370930e+06
US    2.984331e+06
DE    8.121814e+05
dtype: float64

In [19]:
#3 countries with higest per capita footprint
e_CO2_reg = f_sat_CO2 @ L @ Y_reg + F_sat_hh_CO2_reg
e_CO2_reg_pp = e_CO2_reg/pop2019.values/1000000
e_CO2_reg_pp.nlargest(n=3)

region
US    3311.151864
CN    2702.206876
IE    1574.333156
dtype: float64

In [20]:
#trade net importer or not
e_net_import_CO2 = e_CO2_reg - e_CO2_ter
e_net_import_pp_CO2 = e_net_import_CO2/pop2019.values
e_net_import_pp_CO2.loc[["CN", "US", "DE", "IE"]]

region
CN   -6.687232e+08
US    3.268209e+08
DE    3.117986e+08
IE    1.465121e+09
dtype: float64

In [21]:
E_CO2 = np.diag(f_sat_CO2) @ L @ Y_reg
E_CO2.index = Y.index

E_CO2_top = E_CO2/pop2019.values/1000000
E_CO2_top.sum().nlargest(n=3)

region
US    3311.147833
CN    2702.206327
IE    1574.331909
dtype: float64

### Q4.2 Focusing on the two indicators, describe three significantly different findings about the main land use players of the world (15 points)

The description for each finding should:

1) be supported by numerical results you obtained from above calculations and based on comparisons of numerical results; 

2) contain your reflections about the land use stress and problems worldwide


_write your answer here_